In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
import os
key=os.getenv("OPENAI_API_KEY")

In [13]:
llm=ChatOpenAI(
    openai_api_key=key,
    model_name="gpt-4o-mini",
    temperature=0.5
)